In [273]:
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle

x, x_len = np.load('datasets\\data_inkwell_preprocessed\\x.npy'), np.load('datasets\\data_inkwell_preprocessed\\x_len.npy')
c, c_len = np.load('datasets\\data_inkwell_preprocessed\\c.npy'), np.load('datasets\\data_inkwell_preprocessed\\c_len.npy')
x[:, 0, -1] = 0

print(x_len.max())
x.shape, x_len.shape, c.shape, c_len.shape

916


((20771, 1200, 3), (20771,), (20771, 75), (20771,))

In [ ]:
train_len = int(x.shape[0] * .95)
val_len = x.shape[0] - train_len
idxs = np.arange(x.shape[0])
shuffle(idxs)

train_idxs, valid_idxs = sorted(idxs[:train_len]), sorted(idxs[train_len:])
len(np.unique(c[train_idxs]))

In [279]:
#idxs = np.arange(x.shape[0])
#train_idxs, valid_idxs = idxs[:15000], idxs[15000:]

#shuffle(train_idxs), shuffle(valid_idxs)
#train_idxs, valid_idxs = sorted(train_idxs[:9500]), sorted(valid_idxs[:2700])
#len(np.unique(c[train_idxs]))

66

In [280]:
idxs = train_idxs
def create_data_arrays(idxs, setname):
    x_tmp = x[idxs, :x_len.max()]
    lens = x_len[idxs]
    
    strokes = np.zeros_like(x_tmp)
    strokes[:, :, 0], strokes[:, :, 1:] = x_tmp[:, :, -1], x_tmp[:, :, :-1]
        
    #all_strokes = np.vstack([strokes[i, :lens[i]] for i in range(len(idxs))])
    #mu, std = all_strokes.mean(axis=0), all_strokes.std(axis=0)
    #mu[0], std[0] = 0, 1
    
    #strokes = (strokes - mu) / std
    #for i in range(len(strokes)): strokes[i, lens[i]:] = 0

    masks = np.zeros((len(idxs), x_len.max()-1))
    for i in range(len(masks)): masks[i, :lens[i]-1] = 1
    
    labels, label_lengths = c[idxs], c_len[idxs]
    onehots = np.zeros((len(idxs), max(label_lengths), labels.max()))
    for i in range(len(onehots)):
        label = (labels[i]-1)[:label_lengths[i]-1]
        onehots[i, np.arange(len(label)), label] = 1

    assert (np.array([strokes[i][:masks[i].sum().astype(int)+1][-1, 0] for i in range(len(idxs))]) == 1).all()
    
    print(strokes.shape, masks.shape, onehots.shape)
    np.save('datasets\\data_inkwell_preprocessed_subset\\%s_strokes_700.npy' % setname, strokes)
    np.save('datasets\\data_inkwell_preprocessed_subset\\%s_masks_700.npy' % setname, masks)
    np.save('datasets\\data_inkwell_preprocessed_subset\\%s_onehot_700.npy' % setname, onehots)

create_data_arrays(train_idxs, 'train')
create_data_arrays(valid_idxs, 'validation')

(9500, 916, 3) (9500, 915) (9500, 48, 72)
(2700, 916, 3) (2700, 915) (2700, 47, 72)


In [119]:
iam_strokes, iam_masks, iam_one_hots = np.load('datasets\\data_iam_normalized\\train_strokes_700.npy'), np.load('datasets\\data_iam_normalized\\train_masks_700.npy'), np.load('datasets\\data_iam_normalized\\train_onehot_700.npy')
iam_strokes.shape, iam_masks.shape, iam_one_hots.shape

((9500, 701, 3), (9500, 700), (9500, 64, 82))

In [135]:
iam_strokes[0]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.19509466, -0.00296552],
       [ 0.        , -0.21840281,  0.7154464 ],
       ...,
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [173]:
iam_strokes[0][:iam_masks[0].sum().astype(int)+1]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.19509466, -0.00296552],
       [ 0.        , -0.21840281,  0.7154464 ],
       ...,
       [ 0.        ,  0.01467867,  0.44936791],
       [ 0.        , -0.3116354 ,  0.05025018],
       [ 1.        , -0.12517022,  0.10346588]])

In [158]:
(c[0]-1)[:c_len[0]-1]

array([41, 60, 63, 60, 59, 65, 60,  0,  7,  0, 37, 59, 65, 46, 63, 54, 60,
        0, 46, 59, 49], dtype=int8)

In [269]:
import torch

alphabet = [
    ' ', '!', '"', '#', "'", '(', ')', ',', '-', '.',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';',
    '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
    'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
    'y', 'z'
]

char_to_code = {c:i for i,c in enumerate(alphabet)}
code_to_char = {i:c for c,i in char_to_code.items()}
torch.save(char_to_code, 'datasets\\data_inkwell_preprocessed\\char_to_code.pt')

In [267]:
''.join([code_to_char[n] for n in (c[0] - 1)[:c_len[0]-1]])

'Toronto , Ontario and'